# Translation and Summary of Foreign News

The goal of this project will be to get a summary of news in foreign countries by automatically scraping a newspapers headlines or articles, and then translating the text with OpenAI and also providing a summary format.

## Data (Web Scraping)

Let's explore how we could set-up a scraping job (note this has its limits, not every website can be easily scraped, your results may vary on other newpaper sites).

In [23]:
import requests
import bs4
import os
import openai

In [52]:
# This is just a limited example dictionary - We will build this dictionary of the common international news sites
# As each website is different, this gets harder with more countries!
country_newspapers = {"Spain":'https://elpais.com/',
                      "France":"https://www.lemonde.fr/",
                      "Netherlands":"https://www.deutschland.de/de/"
                      }

In [25]:
country = input("What country are you interested in for a news summary? ")

In [26]:
url = country_newspapers[country]
url

'https://www.lemonde.fr/'

### Parsing HTML

In [27]:
result = requests.get(url)
soup = bs4.BeautifulSoup(result.text, "lxml")

In [28]:
soup.select('.article__title-label')

[<p class="article__title-label">Des rescapés du massacre de la rave-party en Israël témoignent : « On ne voyait pas comment s’en tirer »</p>]

In [29]:
soup  # See the entire parsed text

<!DOCTYPE html>
<html lang="fr" prefix="og: http://ogp.me/ns#"> <head> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/> <meta content="no-referrer-when-downgrade" name="referrer"/> <meta content="#ffffff" name="theme-color"/> <script async="1" data-gdpr-purposes="personalization" data-gdpr-src="//www.lemonde.fr/bucket/assets/9b6e1d3a1d035999ff750fe3b08fe717b62420d7/js/chartbeatMab.bundle.js" type="text/plain"></script> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-US" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-CA" rel="alternate"/> <link data-rh="true" href="https://www.lemonde.fr/en/" hreflang="en-GB" rel="alternate"/> <link href="//img.lemde.fr" rel="preconnect"/> <link as="image" fetchpriority="high" imagesizes="100vw

Lets Parse the Netherlands site

In [53]:
result = requests.get(country_newspapers['Netherlands'])
soup = bs4.BeautifulSoup(result.text, 'lxml')
soup

<!DOCTYPE html>
<html dir="ltr" lang="de" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<head>
<meta charset="utf-8"/>
<style>/* @see https://github.com/aFarkas/lazysizes#broken-image-symbol */.js img.lazyload:not([src]) { visibility: hidden; }/* @see https://github.com/aFarkas/lazysizes#automatically-setting-the-sizes-attribute */.js img.lazyloaded[data-sizes=auto] { display: block; width: 100%; }</style>
<meta content="index, follow" name="robots"/>
<meta content="Deutschland verstehen: deutschland.de erklärt Deutschlands Politik ✓, Wirtschaft ✓, Gesellschaft ✓, Kultur ✓ und globale Partnerschaften ✓. Aktuell, übersichtlich und verständlich. ►" name="description

In [54]:
soup.select('.article-teaser__summary')  

[<div class="article-teaser__summary">Der Wald muss sich an  Klimaerwärmung und Extremwetter anpassen. Wir erklären, wie das gelingen kann.  </div>,
 <div class="article-teaser__summary">Weltweit leiden die Wälder unter illegaler Abholzung. Waldexpertin Susanne Gotthardt erklärt, wie der WWF Wälder in Südostasien rettet. </div>,
 <div class="article-teaser__summary">Was braucht der Wald, wenn sich das Klima verändert? Das versucht das Projekt FutureForest mit Hilfe von KI zu ermitteln. </div>,
 <div class="article-teaser__summary">Die Deutschen und ihr Wald – hier liest du einen kleinen Countdown über eine besonders emotionale Beziehung.</div>,
 <div class="article-teaser__summary">Wenn du diese erstaunlichen Fakten kennst, kannst du bei jeder Diskussion über den deutschen Wald mitreden.</div>,
 <div class="article-teaser__summary">Gülsah Wilke hat eine klare Mission: die deutsche Tech-Szene diverser zu machen. Wie sie das mit dem Verein 2hearts schaffen will, lest ihr hier.</div>,
 <d

Now lets parse the spanish site as well

In [38]:
result = requests.get(country_newspapers['Spain'])
soup = bs4.BeautifulSoup(result.text, 'lxml')

In [39]:
len(soup.select('.c_t')) # Check the headlines 

142

In [47]:
soup.select('.c_t')[:3] # Take the first three

[<h2 class="c_t"><a href="https://elpais.com/internacional/2023-10-10/guerra-entre-israel-y-gaza-en-directo.html">Israel recupera el control de la frontera y afirma haber hallado 1.500 cadáveres de milicianos de Hamás</a></h2>,
 <h2 class="c_t"><a href="https://elpais.com/internacional/2023-10-09/israel-cerca-por-completo-gaza-para-asfixiar-a-hamas-tras-16-anos-de-bloqueo.html"><span class="c_t_i c_t_i-s _pr" name="elpais_ico"></span>Asedio total de la Franja para asfixiar a Hamás tras 16 años de bloqueo</a></h2>,
 <h2 class="c_t"><a href="https://elpais.com/internacional/2023-10-09/la-escalada-entre-hamas-e-israel-pone-a-prueba-a-los-tradicionales-mediadores-de-la-region.html"><span class="c_t_i c_t_i-s _pr" name="elpais_ico"></span>La escalada del conflicto pone a prueba a los tradicionales mediadores de la región</a></h2>]

### Grabbing Text

In [48]:
for tag in soup.select('.c_t')[:3]:
    print(tag.text)

Israel recupera el control de la frontera y afirma haber hallado 1.500 cadáveres de milicianos de Hamás
Asedio total de la Franja para asfixiar a Hamás tras 16 años de bloqueo
La escalada del conflicto pone a prueba a los tradicionales mediadores de la región


In [55]:
# We will combine these respective tags and the URL in the dictionary
country_newspapers = {"Spain":('https://elpais.com/','.c_t'),
                      "France":("https://www.lemonde.fr/",'.article__title-label'),
                      "Netherlands":("https://www.deutschland.de/de/",'.article-teaser__summary')
                      }

### Translation via OpenAI

In [59]:
''' Create the prompt '''
def create_prompt():
    # Get Country
    country = input(f"What country would you like the news summary for ?")
    # Get Country's URL newspaper and the respective tag
    try:
        url, tag = country_newspapers[country]
    except:
        print(f"Sorry that country is not supported!!")
        return
    
    # Scrape the website
    results = requests.get(url)
    soup = bs4.BeautifulSoup(results.text, 'lxml')
    
    # Grab all the text together
    country_headline = ""
    for item in soup.select(tag)[:3]:
        country_headline += item.getText()+"\n"
    
    final_prompt = "Detect the language of the news headline below and then translate a summary in English in a conversational tone:\n" + country_headline
    return final_prompt

In [68]:
final_prompt = create_prompt()
print(final_prompt)

Detect the language of the news headline below and then translate a summary in English in a conversational tone:
Israel recupera el control de la frontera de Gaza y afirma haber hallado 1.500 cadáveres de milicianos de Hamás
Asedio total de la Franja para asfixiar a Hamás tras 16 años de bloqueo
La escalada del conflicto pone a prueba a los tradicionales mediadores de la región


In [69]:
# Get the OpenAPI Key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [70]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt = final_prompt,
                                    temperature = 0.1,
                                    max_tokens = 200,
                                    top_p = 1.0,
                                    frequency_penalty=0.0,
                                    presence_penalty=0.0
                                    )

In [71]:
# Check the response
print(response['choices'][0]['text'])


The headline is in Spanish. Translated to English, it reads: Israel has regained control of the Gaza border and claims to have found 1,500 corpses of Hamas militants. A total siege of the Strip to choke Hamas after 16 years of blockade has escalated the conflict, testing the region's traditional mediators.
